In [20]:
import pandas as pd
import numpy as np

from IPython.display import display
import datetime
import plotly.express as px
from plotly.colors import sequential
from plotly.colors import diverging

from plotly.subplots import make_subplots

import math

In [21]:
matches_table = pd.read_csv('data/matches.csv')
display(matches_table)

,Date,Format,Match Number,Stage,Player 1,Player 2,Score 1,Score 2
0,2023-11-18,[Gen 9] OU,1,Group,Evan Sooklal,Will Simpson,2,6
1,2023-11-18,[Gen 9] OU,2,Group,Paul Bartenfeld,Roman Ramirez,2,6
2,2023-11-18,[Gen 9] OU,3,Group,Roman Ramirez,Evan Sooklal,6,3
3,2023-11-18,[Gen 9] OU,4,Group,Will Simpson,Paul Bartenfeld,6,1
4,2023-11-18,[Gen 9] OU,5,Group,Paul Bartenfeld,Evan Sooklal,1,6
...,...,...,...,...,...,...,...,...
69,2024-01-28,[Gen 9] VGC 2024 Reg F,10,Group,Roman Ramirez,Will Simpson,1,4
70,2024-01-28,[Gen 9] VGC 2024 Reg F,11,Group,Evan Sooklal,Roman Ramirez,4,3
71,2024-01-28,[Gen 9] VGC 2024 Reg F,12,Group,Aaron Carter,Will Simpson,2,4
72,2024-01-28,[Gen 9] VGC 2024 Reg F,1,Final,Will Simpson,Aaron Carter,4,0


In [22]:
updated_time = f'<i>Updated {str(datetime.datetime.now().strftime("%A, %b %d, %Y %H:%M:%S"))} CT</i>'
link_fig = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/index.html'>ELO Graph</a>"
link_fig1 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig1.html'>H2H Stats</a>"
link_fig2 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig2.html'>H2H Advanced Stats</a>"
link_fig3 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig3.html'>H2H More Stats</a>"
link_fig4 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig4.html'>Player Data</a>"
link_fig5 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig5.html'>Matches Data</a>"
link_fig6 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig6.html'>Money-Lines</a>"

In [23]:
set_player1 = set(matches_table['Player 1'])
set_player2 = set(matches_table['Player 2'])
set_players = set_player1.union(set_player2)
num_players = len(set_players)
display(set_players)

{'Aaron Carter',
 'Evan Sooklal',
 'Paul Bartenfeld',
 'Roman Ramirez',
 'Will Simpson'}

In [24]:
# elo stuff
K = 31
base = 3

def expected_score(ratingA, ratingB, base=None):
    # game variables
    # base = 10
    normal_elo_difference = 100

    return (1 / (1 + np.power(base, (ratingB - ratingA) / normal_elo_difference)))

def rating_change(score, expected_score, K=None):
    # k-factor: determines how strongly a result affects the rating change
    # usually between 10 and 40, bit with a lot of games, we want to change it often
    # K = 32
    return K * (score - expected_score)

def sort_index_by_elo(table, elo):
    for i in range(2):
        table.sort_index(axis=i, key=(lambda x: [elo[y] for y in x.values]), inplace=True)

def add_elo_to_index(table):
    table.index= [f"{x}<br>{(elo[x]):,.0f}" for x in table.index]
    table.columns = [f"{x}<br>({(elo[x]):,.0f})" for x in table.columns]

def generate_elo(K, base):
    # ELO INITIALIZATION
    starting_elo = 1200.0
    elo = dict.fromkeys(set_players, starting_elo)
    elo_time = np.zeros([len(matches_table) + 1, num_players])
    elo_time_table = pd.DataFrame(elo_time)
    elo_time_table.columns = sorted(elo)
    elo_time_table.replace(0, np.NaN, inplace=True)

    prev_elo_time_table = elo_time_table.copy(deep=True)
    d_elo_time_table = elo_time_table.copy(deep=True)
    exp_elo_time_table = elo_time_table.copy(deep=True)
    wl_time_table = elo_time_table.copy(deep=True)

    record_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]),dtype=object)
    record_table.index = list(set_players)
    record_table.columns = list(set_players)

    luck_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]))
    luck_table.index = list(set_players)
    luck_table.columns = list(set_players)

    for c in record_table.columns:
        for r in record_table.index:
            record_table.at[r, c] = np.array([0, 0])

    plus_minus_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]),dtype=object)
    plus_minus_table.index = list(set_players)
    plus_minus_table.columns = list(set_players)
    
    for c in plus_minus_table.columns:
        for r in plus_minus_table.index:
            plus_minus_table.at[r, c] = np.array([0, 0])

    elo_time_table.iloc[0,:] = starting_elo
    prev_elo_time_table.iloc[0,:] = starting_elo

    for (i, row) in matches_table.iterrows():
        elo_p1 = elo[row['Player 1']]
        elo_p2 = elo[row['Player 2']]

        prev_elo_time_table.loc[i + 1, row['Player 1']] = elo_p1
        prev_elo_time_table.loc[i + 1, row['Player 2']] = elo_p2
        
        win_prob_p1 = expected_score(elo_p1, elo_p2, base=base)
        win_prob_p2 = expected_score(elo_p2, elo_p1, base=base)
        
        exp_elo_time_table.loc[i + 1, row['Player 1']] = win_prob_p1
        exp_elo_time_table.loc[i + 1, row['Player 2']] = win_prob_p2

        rating_change_p1 = rating_change(row['Score 1'] > row['Score 2'], win_prob_p1, K=K)
        rating_change_p2 = rating_change(row['Score 2'] > row['Score 1'], win_prob_p2, K=K)

        if rating_change_p1 > 0:
            luck_table.loc[row['Player 1'], row['Player 2']] += 1 - win_prob_p1
            luck_table.loc[row['Player 2'], row['Player 1']] += 0 - win_prob_p2
        elif rating_change_p2 > 0:
            luck_table.loc[row['Player 1'], row['Player 2']] += 0 - win_prob_p1
            luck_table.loc[row['Player 2'], row['Player 1']] += 1 - win_prob_p2

        plus_minus_table.loc[row['Player 1'], row['Player 2']][0] += row['Score 1']
        plus_minus_table.loc[row['Player 1'], row['Player 2']][1] += row['Score 2']
        plus_minus_table.loc[row['Player 2'], row['Player 1']][0] += row['Score 2']
        plus_minus_table.loc[row['Player 2'], row['Player 1']][1] += row['Score 1']

        if row['Score 1'] > row['Score 2']:
            record_table.loc[row['Player 1'], row['Player 2']][0] += 1
            record_table.loc[row['Player 2'], row['Player 1']][1] += 1
            wl_time_table.loc[i + 1, row['Player 1']] = row['Score 1'] - row['Score 2']
            wl_time_table.loc[i + 1, row['Player 2']] = row['Score 2'] - row['Score 1']
        elif row['Score 2'] > row['Score 1']:
            record_table.loc[row['Player 2'], row['Player 1']][0] += 1
            record_table.loc[row['Player 1'], row['Player 2']][1] += 1
            wl_time_table.loc[i + 1, row['Player 1']] = row['Score 1'] - row['Score 2']
            wl_time_table.loc[i + 1, row['Player 2']] = row['Score 2'] - row['Score 1']

        d_elo_time_table.loc[i + 1, row['Player 1']] = rating_change_p1
        d_elo_time_table.loc[i + 1, row['Player 2']] = rating_change_p2

        elo[row['Player 1']] += rating_change_p1
        elo[row['Player 2']] += rating_change_p2

        elo_time_table.loc[i + 1, row['Player 1']] = elo[row['Player 1']]
        elo_time_table.loc[i + 1, row['Player 2']] = elo[row['Player 2']]

    prev_elo_time_table = prev_elo_time_table.iloc[1:,:]
    elo_time_table = elo_time_table.iloc[1:,:]
    d_elo_time_table = d_elo_time_table.iloc[1:,:]
    exp_elo_time_table = exp_elo_time_table.iloc[1:,:]
    wl_time_table = wl_time_table.iloc[1:,:]

    list_players = sorted(set_players, key=lambda x: elo[x], reverse=False)

    for table in [record_table, luck_table, plus_minus_table]:
        sort_index_by_elo(table, elo)

    return (elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, wl_time_table, record_table, luck_table, plus_minus_table)

(elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, wl_time_table, record_table, luck_table, plus_minus_table) = generate_elo(K=K,base=base)

luck_per_game_table = luck_table.copy(deep=True)

for c in luck_per_game_table.columns:
    for r in luck_per_game_table.index:
        if r !=c :
            luck_per_game_table.loc[r, c] /= record_table.loc[r, c].sum()
        else:
            luck_per_game_table.loc[r, c] = np.NaN


display(record_table)
display(luck_per_game_table)
display(plus_minus_table)
display(wl_time_table)

prev_elo_time_table.to_excel('exports/prev_elo_time_table.xlsx')
d_elo_time_table.to_excel('exports/d_elo_time_table.xlsx')
elo_time_table.to_excel('exports/elo_time_table.xlsx')
exp_elo_time_table.to_excel('exports/exp_elo_time_table.xlsx')



,Paul Bartenfeld,Roman Ramirez,Evan Sooklal,Aaron Carter,Will Simpson
Paul Bartenfeld,"[0, 0]","[2, 4]","[1, 5]","[1, 3]","[0, 6]"
Roman Ramirez,"[4, 2]","[0, 0]","[1, 7]","[1, 5]","[6, 5]"
Evan Sooklal,"[5, 1]","[7, 1]","[0, 0]","[3, 6]","[2, 8]"
Aaron Carter,"[3, 1]","[5, 1]","[6, 3]","[0, 0]","[4, 4]"
Will Simpson,"[6, 0]","[5, 6]","[8, 2]","[4, 4]","[0, 0]"


,Paul Bartenfeld,Roman Ramirez,Evan Sooklal,Aaron Carter,Will Simpson
Paul Bartenfeld,NaN,-0.054102,-0.179256,-0.061861,-0.301516
Roman Ramirez,0.054102,NaN,-0.361600,-0.191107,0.118365
Evan Sooklal,0.179256,0.361600,NaN,-0.060280,-0.262323
Aaron Carter,0.061861,0.191107,0.060280,NaN,-0.010279
Will Simpson,0.301516,-0.118365,0.262323,0.010279,NaN


,Paul Bartenfeld,Roman Ramirez,Evan Sooklal,Aaron Carter,Will Simpson
Paul Bartenfeld,"[0, 0]","[21, 31]","[24, 35]","[15, 21]","[16, 36]"
Roman Ramirez,"[31, 21]","[0, 0]","[27, 41]","[20, 26]","[50, 46]"
Evan Sooklal,"[35, 24]","[41, 27]","[0, 0]","[33, 46]","[28, 51]"
Aaron Carter,"[21, 15]","[26, 20]","[46, 33]","[0, 0]","[31, 27]"
Will Simpson,"[36, 16]","[46, 50]","[51, 28]","[27, 31]","[0, 0]"


,Aaron Carter,Evan Sooklal,Paul Bartenfeld,Roman Ramirez,Will Simpson
1,NaN,-4.0,NaN,NaN,4.0
2,NaN,NaN,-4.0,4.0,NaN
3,NaN,-3.0,NaN,3.0,NaN
4,NaN,NaN,-5.0,NaN,5.0
5,NaN,5.0,-5.0,NaN,NaN
...,...,...,...,...,...
70,NaN,NaN,NaN,-3.0,3.0
71,NaN,1.0,NaN,-1.0,NaN
72,-2.0,NaN,NaN,NaN,2.0
73,-4.0,NaN,NaN,NaN,4.0


In [25]:
# player stats

player_table = pd.DataFrame(np.zeros([len(set_players), 0]))
player_table.index = set_players
player_table.sort_index(axis=0, key=(lambda x: [elo[y] for y in x.values]), ascending=False, inplace=True)

# ELO

player_table['Current ELO'] = [elo[x] for x in player_table.index]
player_table['Max ELO'] = elo_time_table.replace(to_replace=np.NaN, value=-np.inf).max(axis=0)
player_table['Min ELO'] = elo_time_table.replace(to_replace=np.NaN, value=np.inf).min(axis=0)

date2matches_len = dict.fromkeys(set(matches_table['Date']),0)
date2matches_first = dict.fromkeys(set(matches_table['Date']),0)
for (i, row) in matches_table.iterrows():
    if date2matches_len[row['Date']] == 0:
        date2matches_first[row['Date']] = i
    date2matches_len[row['Date']] += 1

# CHAMPIONSHIPS AND RUNNER UPS

championships = dict.fromkeys(set_players, 0)
runner_ups = dict.fromkeys(set_players, 0)
for date in set(matches_table['Date']):
    row = matches_table.iloc[date2matches_first[date] + date2matches_len[date] - 1, :]
    if row['Score 1'] > row['Score 2']:
        championships[row['Player 1']] += 1
        runner_ups[row['Player 2']] += 1
    elif row['Score 2'] > row['Score 1']:
        championships[row['Player 2']] += 1
        runner_ups[row['Player 1']] += 1

player_table['Titles'] = pd.DataFrame(championships.values(), index=championships.keys())
player_table['Runner-Ups'] = pd.DataFrame(runner_ups.values(), index=runner_ups.keys())

# RECORDS

player_table['Record'] = record_table.sum(axis=1)
player_table['GP'] = [sum(r) for r in player_table['Record']]
player_table['W'] = [r[0] for r in player_table['Record']]
player_table['L'] = [r[1] for r in player_table['Record']]
player_table['WL'] = player_table['W'] - player_table['L']
player_table['GB'] = (player_table['WL'] - player_table['WL'].max()) / 2
player_table['PCT'] = player_table['W'] / player_table['GP']
player_table.drop(columns='Record', inplace=True)

# POINTS

player_table['Raw +/-'] = plus_minus_table.sum(axis=1)
player_table['PD'] = [r[0] - r[1] for r in player_table['Raw +/-']]
player_table['PF'] = [r[0] for r in player_table['Raw +/-']]
player_table['PA'] = [r[1] for r in player_table['Raw +/-']]
player_table['PD/G'] = player_table['PD'] / player_table['GP']
player_table['PF/G'] = player_table['PF'] / player_table['GP']
player_table['PA/G'] = player_table['PA'] / player_table['GP']
player_table.drop(columns='Raw +/-', inplace=True)

# LUCK

player_table['Luck'] = luck_table.sum(axis=1)
player_table['Luck/G'] = player_table['Luck'] / player_table['GP']

# N-POINT GAMES
npg_table = pd.DataFrame(np.zeros([len(set_players), 0]))
npg_table.index = player_table.index
sort_index_by_elo(npg_table, elo)

for i in range(6):
    npg_table[f'{i+1}-PT G'] = [[0, 0] for x in range(len(set_players))]
for (i, row) in matches_table.iterrows():
    npg_table.loc[row['Player 1'], f"{abs(row['Score 1'] - row['Score 2'])}-PT G"][row['Score 1'] < row['Score 2']] += 1
    npg_table.loc[row['Player 2'], f"{abs(row['Score 1'] - row['Score 2'])}-PT G"][row['Score 1'] > row['Score 2']] += 1

# STREAKS

streaks = dict.fromkeys(set_players, list())

for player in streaks.keys():
    running = [0]
    for is_win in wl_time_table[player].dropna().reset_index(drop=True).apply(np.sign):
        if running[-1] == 0:
            running.append(is_win)
        elif (is_win > 0) and (running[-1] < 0):
            running.append(1)
        elif (is_win < 0) and (running[-1] > 0):
            running.append(-1)
        else:
            running.append(running[-1] + is_win)
    streaks[player] = running[1:]

streaks_table = pd.DataFrame(np.zeros([max([len(x) for x in streaks.values()]), len(set_players)]))
streaks_table.columns = set_players
streaks_table.sort_index(axis=1, key=(lambda x: [elo[y] for y in x.values]), ascending=False, inplace=True)

for player in streaks_table.columns:
    streaks_table[player] = pd.Series(streaks[player])

for x in ['1-PT G', '6-PT G']:
    player_table[x] = npg_table[x]
    player_table[x] = [f'{w}-{l}' for (w, l) in player_table[x]]


player_table['Streak'] = 0
for player in player_table.index:
    val = ""
    if streaks[player][-1] > 0:
        val += "W"
    elif streaks[player][-1] < 0:
        val += "L"
    val += str(int(np.abs(streaks[player][-1])))
    player_table.loc[player, 'Streak'] = val

player_table['L10'] = ""
for player in player_table.index:
    l10_w = 0
    l10_l = 0
    for i in range(10):
        if streaks[player][-1 - i] > 0:
            l10_w += 1
        elif streaks[player][-1 - i] < 0:
            l10_l += 1
    player_table.loc[player, 'L10'] = f"{l10_w}-{l10_l}"

player_table.drop('WL', axis=1, inplace=True)
player_table.drop('Luck', axis=1, inplace=True)

# STYLIZING

from pandas.io.formats.style import Styler

player_table_styled = Styler(player_table, cell_ids=False).format(
    {'Current ELO': '{:.0f}',
    'Max ELO': '{:.0f}',
    'Min ELO': '{:.0f}',
    'WL': '{:+,.0f}',
    'GB': '{:,.1f}',
    'PCT': '{:,.3f}',
    'PD': '{:+.0f}',
    'PD/G': '{:+,.1f}',
    'PF/G': '{:,.1f}',
    'PA/G': '{:,.1f}',
    'Luck': '{:+,.1%}',
    'Luck/G': '{:+,.1%}'
    }
)

player_table_styled.set_caption("Player Table Statistics")

styles = [
    {
        'selector': '',
        'props': [
            ('text-align', 'center'),
            ('font-size', '11pt'),
            ('border-collapse', 'collapse'),#'separated'),
            ('border', '1px solid silver')
        ]
    },
    {
        'selector': 'caption',
        'props': [('font', 'bold 20px Verdana')]
    },
    {
        'selector': 'tr:nth-child(even)',
        'props': 'background-color: #D3D3D3'
    },
    {
        'selector': 'td:hover',
        'props': [('background-color','#ffffb3')]
    },
    {
        'selector': 'th',
        'props': [
            ('font-family', 'Verdana'),
            ('background-color', '#828282'),
            ('color', '#FFFFFF'),
            ('padding-left', '5px'),
            ('padding-right', '5px')
        ]
    },
    {
        'selector': 'td',
        'props': [
            ('font-family', 'Courier New'),
            ('font-weight', 'bold'),
            ('padding-left', '5px'),
            ('padding-right', '5px')
        ]
    }
]

player_table_styled.set_table_styles(styles, overwrite=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(player_table_styled)

player_table_styled.to_html('figures/fig4.html')

matches_table_styled = Styler(matches_table, cell_ids=False)
matches_table_styled.set_caption("Matches Table")
matches_table_styled.set_table_styles(styles, overwrite=True)
matches_table_styled.to_html('figures/fig5.html')



,Current ELO,Max ELO,Min ELO,Titles,Runner-Ups,GP,W,L,GB,PCT,PD,PF,PA,PD/G,PF/G,PA/G,Luck/G,1-PT G,6-PT G,Streak,L10
Will Simpson,1300,1300,1197,2,1,35,23,12,0.0,0.657,+35,160,125,+1.0,4.6,3.6,+9.2%,3-1,0-1,W5,8-2
Aaron Carter,1257,1310,1182,1,1,27,18,9,-1.0,0.667,+29,124,95,+1.1,4.6,3.5,+6.9%,4-3,1-1,L3,6-4
Evan Sooklal,1225,1252,1172,0,2,33,17,16,-5.0,0.515,-11,137,148,-0.3,4.2,4.5,+2.4%,7-4,0-0,W1,4-6
Roman Ramirez,1125,1258,1125,1,0,31,12,19,-9.0,0.387,-6,128,134,-0.2,4.1,4.3,-7.8%,0-5,1-0,L7,2-8
Paul Bartenfeld,1093,1184,1093,0,0,22,4,18,-12.5,0.182,-47,76,123,-2.1,3.5,5.6,-15.7%,1-2,0-0,L8,2-8


In [40]:
import plotly.graph_objects as go
import plotly.colors as pc

fig = go.Figure()

# adding player
ranking = 1
trace_colors = pc.qualitative.Bold
for (i, (player, current_rating)) in enumerate(sorted(elo.items(), key=lambda x:x[1], reverse=True)):
    fig.add_trace(go.Scatter(
        x=elo_time_table.index,
        y=elo_time_table[player],
        name=f'#{ranking} ({current_rating:.0f}) {player}',
        mode='lines+markers',
        connectgaps=True,
        text=[
            f"<br><b>Pre-Game ELO: </b>{x[1]:.0f}<br><b>Win Probability: </b>{x[0]:.1%}<br><br><b>Change in ELO:</b> {x[2]:+.0f}<br>" for x in zip(
                exp_elo_time_table[player],
                prev_elo_time_table[player],
                d_elo_time_table[player]
            )
        ],
        line=dict(
            shape='hv',
            color=trace_colors[i % len(trace_colors)]
        )
    ))
    ranking += 1

# adding highlighting by tournament
tournaments = list(sorted(set(matches_table['Date'])))
vrect_colors = ['green', 'red', 'yellow', 'blue', 'orange']
for (i, tourney) in enumerate(tournaments):
    fig.add_vrect(
        annotation_text=tourney,
        annotation_position="top left",
        x0=matches_table['Date'][matches_table['Date'] == tourney].index[0] + 0.5,
        x1=matches_table['Date'][matches_table['Date'] == tourney].index[-1] + 1.5,
        fillcolor=vrect_colors[i % len(vrect_colors)],
        opacity=0.1,
        line_width=0
    )

fig.update_layout(
    title=f'<b>Pokémon Showdown ELO Rating System by Roman Ramirez</b><br>{updated_time}<br>{link_fig1}, {link_fig2}, {link_fig3}, {link_fig4}, {link_fig5}, {link_fig6}',
    xaxis_title='<b>Game Number</b>',
    yaxis_title='<b>ELO Rating</b>'
)

customdata = np.stack((
        list(matches_table['Player 1']),
        list(matches_table['Player 2']),
        list(matches_table['Score 1']),
        list(matches_table['Score 2']),
        matches_table['Date'],
        list(matches_table['Format']),
        list(matches_table['Match Number'])
    ), axis=-1)
hovertemplate = (
    '<i>%{customdata[4]|%A, %B %d, %Y}, Game %{x}</i><br>' +
    '<b>%{fullData.name}</b><br><br>' + 
    '<b>Format:</b> %{customdata[5]}<br>' + 
    '<b>Tournament Match:</b> %{customdata[6]}<br><br>' + 
    '<b>%{customdata[0]} vs. %{customdata[1]}</b><br>' +
    '<b>Final Score:</b> %{customdata[2]}-%{customdata[3]}<br>' + 
    '%{text}' + 
    '<b>Post-Game ELO:</b> %{y:,.0f}<br>' +
    '<extra></extra>'
)

fig.update_traces(
    customdata=customdata,
    hovertemplate=hovertemplate,
    opacity=0.8,
    legendgrouptitle_text='<b>#<i>Rank</i> (<i>Current ELO</i>) <i>Player</i></b>'
)

fig.show()
fig.write_html("index.html")

In [27]:
# historical table, with record_table
win_pct_table = record_table.copy(deep=True)
text_record_table = record_table.copy(deep=True)

for c in win_pct_table.columns:
    for r in win_pct_table.index:
        if c == r:
            win_pct_table.loc[r, c] = np.NaN
            text_record_table.loc[r, c] = ""
        else:
            w, l = win_pct_table.loc[r, c]
            w_pct = (w) / (w + l) if (w + l) != 0 else 0
            win_pct_table.loc[r, c] = w_pct
            text_record_table.loc[r, c] = f"{w}-{l}<br>{w_pct:,.1%}"

win_prob_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]), dtype=object)
win_prob_table.columns = set_players
win_prob_table.index = set_players
sort_index_by_elo(win_prob_table, elo)

for c in win_prob_table.columns:
    for r in win_prob_table.index:
            if r != c:
                win_prob_table.loc[r, c] = expected_score(elo[r], elo[c], base=base)
            else:
                win_prob_table.loc[r, c] = np.NaN


In [28]:
fig1 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[f"<b>{x}</b>" for x in ["Historical Win Percentage", "Current Matchup Predictor"]],
    x_title=f"{link_fig}, {link_fig2}, {link_fig3}, {link_fig4}, {link_fig5}, {link_fig6}"
)

go_win_pct = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=win_pct_table,
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=text_record_table,
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

go_win_prob = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=win_prob_table,
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=win_prob_table.applymap(lambda x: "" if np.isnan(x) else f"{x:.1%}"),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

fig1.add_trace(go_win_pct, row=1, col=1)
fig1.add_trace(go_win_prob, row=1, col=2)

fig1.layout.yaxis1.title="<b>Self</b>"
fig1.layout.yaxis2.title="<b>Self</b>"
fig1.layout.xaxis1.title="<b>Other</b>"
fig1.layout.xaxis2.title="<b>Other</b>"

fig1.update_traces(hoverinfo='skip')

fig1.update_layout(
    title=f"<b>Head-to-Head Statistics</b>, {updated_time}"
)

fig1.show()
fig1.write_html('figures/fig1.html')

In [29]:
fig3 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[f"<b>{x}</b>" for x in ["Head-to-Head Plus Minus", "N-Game Historical Record"]],
    x_title=f"<br>{link_fig}, {link_fig1}, {link_fig2}, {link_fig4}, {link_fig5}, {link_fig6}"
)

text_npg_table = npg_table.copy(deep=True)

for c in npg_table.columns:
    for r in npg_table.index:
        if c == r:
            text_npg_table.loc[r, c] = ""
        else:
            w, l = npg_table.loc[r, c]
            if (w + l) == 0:
                text_npg_table.loc[r, c] = ""
            else:
                w_pct = (w) / (w + l) if (w + l) != 0 else 0
                text_npg_table.loc[r, c] = f"{w}-{l}<br>{w_pct:,.1%}"

def calc_win_pct(wl):
    if sum(wl) == 0:
        return np.NaN
    else:
        return (wl[0]) / (sum(wl))

go_ng = go.Heatmap(
    x=[x.replace(" ", "<br>")+"ames" for x in npg_table.columns],
    y=[x.replace(" ", "<br>") for x in npg_table.index],
    z=npg_table.applymap(calc_win_pct),
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=text_npg_table,
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

text_plus_minus_table = plus_minus_table.copy(deep=True)

for c in plus_minus_table.columns:
    for r in plus_minus_table.index:
        if c == r:
            text_plus_minus_table.loc[r, c] = ""
        else:
            w, l = plus_minus_table.loc[r, c]
            if (w + l) == 0:
                text_plus_minus_table.loc[r, c] = ""
            else:
                w_pct = (w) / (w + l) if (w + l) != 0 else 0
                text_plus_minus_table.loc[r, c] = f"{w}-{l}<br>{w_pct:,.1%}"

go_pm = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in plus_minus_table.columns],
    y=[x.replace(" ", "<br>") for x in plus_minus_table.index],
    z=plus_minus_table.applymap(calc_win_pct),
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=text_plus_minus_table,
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

fig3.add_trace(go_pm, row=1, col=1)
fig3.add_trace(go_ng, row=1, col=2)

fig3.layout.yaxis1.title="<b>Self</b>"
fig3.layout.yaxis2.title="<b>Self</b>"
fig3.layout.xaxis1.title="<b>Other</b>"
fig3.layout.xaxis2.title="<b>Other</b>"

fig3.update_traces(
    hoverinfo='skip'
)

fig3.update_layout(
    title=f"<b>Head-to-Head Statistics</b>, {updated_time}"
)

fig3.show()
fig3.write_html('figures/fig3.html')

In [30]:
# difference in win percentage and matchup predictor
# positive number means overrated, negative number means underrated
# understanding: Player A is overrated/underrated against Player B

key_upset_table = -win_pct_table + win_prob_table

fig2 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[
        "<b>Over/Underrated</b><br>How Much Better is the Win Probability Against Record", 
        "<b>Average Luck Per Game</b><br>Average Difference in Actual and Expected Performance"
    ],
    x_title=f"{link_fig}, {link_fig1}, {link_fig3}, {link_fig4}, {link_fig5}, {link_fig6}"
)

go_ou = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=key_upset_table,
    zmax=max(key_upset_table.max(axis=None), luck_per_game_table.max(axis=None)),
    zmid=0,
    zmin=min(key_upset_table.min(axis=None), luck_per_game_table.min(axis=None)),
    text=key_upset_table.applymap(lambda x: "" if np.isnan(x) else f"{x:+.1%}"),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

go_luck_pg = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=luck_per_game_table,
    zmax=max(key_upset_table.max(axis=None), luck_per_game_table.max(axis=None)),
    zmid=0,
    zmin=min(key_upset_table.min(axis=None), luck_per_game_table.min(axis=None)),
    text=luck_per_game_table.applymap(lambda x: "" if np.isnan(x) else f"{x:+.1%}"),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

fig2.add_trace(go_ou, row=1, col=1)
fig2.add_trace(go_luck_pg, row=1, col=2)

fig2.layout.yaxis1.title="<b>Self</b>"
fig2.layout.yaxis2.title="<b>Self</b>"
fig2.layout.xaxis1.title="<b>Other</b>"
fig2.layout.xaxis2.title="<b>Other</b>"

fig2.update_traces(hoverinfo='skip')

fig2.update_layout(
    title=f"<b>Head-to-Head Advanced Statistics</b>, {updated_time}"
)

fig2.show()
fig2.write_html('figures/fig2.html')

In [31]:
# betting/gambling table

# to-do
# odds to win championship
# record for/against the spread
# player vs. player spread
# over/under total

odds_table = win_prob_table / win_prob_table.T
ml_table = odds_table.copy(deep=True)

for c in ml_table.columns:
    for r in ml_table.index:
        ml_table.loc[r, c] = np.NaN
        win_prob = win_prob_table.loc[r, c]
        if r == c:
            pass
        elif win_prob > 0.5:
            ml_table.loc[r, c] = -1 * win_prob / (1 - win_prob) * 100
        elif win_prob < 0.5:
            ml_table.loc[r, c] = (1 - win_prob) / win_prob * 100

ml_table = ml_table.applymap(lambda x: '{:+.0f}'.format((x * 2).round(-1) / 2) if not np.isnan(x) else np.NaN)

display(odds_table)
display(ml_table)

,Paul Bartenfeld,Roman Ramirez,Evan Sooklal,Aaron Carter,Will Simpson
Paul Bartenfeld,NaN,0.700811,0.234434,0.163853,0.103174
Roman Ramirez,1.426919,NaN,0.334518,0.233805,0.14722
Evan Sooklal,4.265599,2.989377,NaN,0.69893,0.440097
Aaron Carter,6.103042,4.277077,1.430759,NaN,0.629673
Will Simpson,9.692401,6.792537,2.272225,1.588126,NaN


,Paul Bartenfeld,Roman Ramirez,Evan Sooklal,Aaron Carter,Will Simpson
Paul Bartenfeld,NaN,+145,+425,+610,+970
Roman Ramirez,-145,NaN,+300,+430,+680
Evan Sooklal,-425,-300,NaN,+145,+225
Aaron Carter,-610,-430,-145,NaN,+160
Will Simpson,-970,-680,-225,-160,NaN


In [32]:
# best 2 out of 3 calculator
best_23_table = win_pct_table.copy(deep=True)
best_23_table = best_23_table.applymap(lambda x: 0)

for c in best_23_table.columns:
    for r in best_23_table.columns:
        if r == c:
            best_23_table.loc[r, c] = np.NaN
        else:
            win_prob = win_prob_table.loc[r, c]
            best_23_table.loc[r, c] = np.power(win_prob, 2) * (1 + (2 * (1 - win_prob)))

display(best_23_table)

,Paul Bartenfeld,Roman Ramirez,Evan Sooklal,Aaron Carter,Will Simpson
Paul Bartenfeld,NaN,0.369428,0.094501,0.053880,0.024604
Roman Ramirez,0.630572,NaN,0.157000,0.094120,0.045178
Evan Sooklal,0.905499,0.843000,NaN,0.368483,0.223096
Aaron Carter,0.946120,0.905880,0.631517,NaN,0.332503
Will Simpson,0.975396,0.954822,0.776904,0.667497,NaN


In [33]:
# making playoffs odds
# assumptions
## assume ELO doesn't change after each game
## assume if you win at least 6/8 of your games, you're in for the championship
## independent on the order of matches
def prob_of_x_wins(player, num_wins=6):
    sp = set_players.copy()
    sp.remove(player)
    schedule = list(sp) + list(sp)

    total_prob = 0
    for i in range(np.power(2, 8)):
        single_prob = 1
        record = '{0:08b}'.format(i)
        if record.count('1') != (num_wins):
            continue
        for (j, x) in enumerate(record):
            if x == '1':
                single_prob *= win_prob_table.loc[player, schedule[j]]
            elif x == '0':
                single_prob *= win_prob_table.loc[schedule[j], player]
        total_prob += single_prob
    return total_prob

prob_of_x_wins_table = pd.DataFrame(np.zeros([len(set_players), 9]))
prob_of_x_wins_table.index = list(set_players)
prob_of_x_wins_table.columns
prob_of_x_wins_table.sort_index(axis=0, key=(lambda x: [elo[y] for y in x.values]), ascending=False, inplace=True)

for c in prob_of_x_wins_table.columns:
    for r in prob_of_x_wins_table.index:
        prob_of_x_wins_table.loc[r, c] = prob_of_x_wins(r, int(c))

odds_of_x_wins_table = (1 - prob_of_x_wins_table) / prob_of_x_wins_table
ml_of_x_wins_table = odds_of_x_wins_table.applymap(lambda x: '{:+,.0f}'.format(float('%.2g' % (x * 100)) if not np.isnan(x) else np.NaN))

# what is the probability of getting at least x wins?
prob_of_cum_x_wins_table = prob_of_x_wins_table.copy(deep=True)
prob_of_cum_x_wins_table.applymap(lambda x: 0)

for (i, c) in enumerate(prob_of_cum_x_wins_table.columns):
    prob_of_cum_x_wins_table.loc[:, c] = prob_of_x_wins_table.iloc[:, i:8+1].sum(axis=1)
prob_of_cum_x_wins_table.drop(columns=0, axis=1, inplace=True)

odds_of_cum_x_wins_table = (1 - prob_of_cum_x_wins_table) / prob_of_cum_x_wins_table
ml_of_cum_x_wins_table = odds_of_cum_x_wins_table.copy(deep=True)

for c in ml_of_cum_x_wins_table.columns:
    for r in ml_of_cum_x_wins_table.index:
        ml_of_cum_x_wins_table.loc[r, c] = np.NaN
        odds = odds_of_cum_x_wins_table.loc[r, c]
        if odds < 1:
            ml_of_cum_x_wins_table.loc[r, c] = -1 * (1 / odds_of_cum_x_wins_table.loc[r, c]) * 100
        elif odds > 1:
            ml_of_cum_x_wins_table.loc[r, c] = odds_of_cum_x_wins_table.loc[r, c] * 100

ml_of_cum_x_wins_table = ml_of_cum_x_wins_table.applymap(lambda x: '{:+,.0f}'.format(float('%.2g' % (x)) if not np.isnan(x) else np.NaN))

display(prob_of_x_wins_table)
display(odds_of_x_wins_table)
display(ml_of_x_wins_table)
display(prob_of_cum_x_wins_table)
display(odds_of_cum_x_wins_table)
display(ml_of_cum_x_wins_table)

,0,1,2,3,4,5,6,7,8
Will Simpson,0.000002,0.000082,0.001366,0.012135,0.062154,0.187068,0.324550,0.299290,0.113353
Aaron Carter,0.000045,0.001129,0.011409,0.060338,0.179596,0.303688,0.284027,0.134684,0.025084
Evan Sooklal,0.000379,0.006356,0.042819,0.148206,0.281589,0.295449,0.169616,0.049762,0.005824
Roman Ramirez,0.047583,0.203891,0.330987,0.264261,0.117589,0.030649,0.004648,0.000379,0.000013
Paul Bartenfeld,0.137616,0.330903,0.317524,0.159024,0.046069,0.007998,0.000820,0.000046,0.000001


,0,1,2,3,4,5,6,7,8
Will Simpson,497925.666299,12235.903359,731.142583,81.404534,15.089083,4.345651,2.081188,2.341243,7.821978
Aaron Carter,22046.537048,885.115845,86.646562,15.573267,4.568040,2.292858,2.520797,6.424765,38.866731
Evan Sooklal,2640.437478,156.340746,22.353918,5.747357,2.551273,2.384680,4.895684,19.095536,170.693649
Roman Ramirez,20.015796,3.904590,2.021270,2.784138,7.504178,31.627130,214.151908,2636.279023,77850.427471
Paul Bartenfeld,6.266599,2.022031,2.149365,5.288377,20.706580,124.036573,1219.122663,21917.972975,941987.072493


,0,1,2,3,4,5,6,7,8
Will Simpson,"+50,000,000","+1,200,000","+73,000","+8,100","+1,500",+430,+210,+230,+780
Aaron Carter,"+2,200,000","+89,000","+8,700","+1,600",+460,+230,+250,+640,"+3,900"
Evan Sooklal,"+260,000","+16,000","+2,200",+570,+260,+240,+490,"+1,900","+17,000"
Roman Ramirez,"+2,000",+390,+200,+280,+750,"+3,200","+21,000","+260,000","+7,800,000"
Paul Bartenfeld,+630,+200,+210,+530,"+2,100","+12,000","+120,000","+2,200,000","+94,000,000"


,1,2,3,4,5,6,7,8
Will Simpson,0.999998,0.999916,0.998550,0.986415,0.924261,0.737193,0.412643,0.113353
Aaron Carter,0.999955,0.998826,0.987417,0.927079,0.747482,0.443795,0.159768,0.025084
Evan Sooklal,0.999621,0.993266,0.950446,0.802240,0.520651,0.225202,0.055587,0.005824
Roman Ramirez,0.952417,0.748526,0.417539,0.153278,0.035689,0.005040,0.000392,0.000013
Paul Bartenfeld,0.862384,0.531481,0.213956,0.054933,0.008864,0.000866,0.000047,0.000001


,1,2,3,4,5,6,7,8
Will Simpson,0.000002,0.000084,0.001452,0.013772,0.081945,0.356496,1.423402,7.821978
Aaron Carter,0.000045,0.001175,0.012744,0.078657,0.337825,1.253295,5.259077,38.866731
Evan Sooklal,0.000379,0.006780,0.052137,0.246509,0.920672,3.440454,16.989940,170.693649
Roman Ramirez,0.049961,0.335959,1.394983,5.524072,27.019648,197.416536,2549.866395,77850.427471
Paul Bartenfeld,0.159576,0.881536,3.673848,17.204023,111.816726,1153.369323,21419.541743,941987.072493


,1,2,3,4,5,6,7,8
Will Simpson,"-50,000,000","-1,200,000","-69,000","-7,300","-1,200",-280,+140,+780
Aaron Carter,"-2,200,000","-85,000","-7,800","-1,300",-300,+130,+530,"+3,900"
Evan Sooklal,"-260,000","-15,000","-1,900",-410,-110,+340,"+1,700","+17,000"
Roman Ramirez,"-2,000",-300,+140,+550,"+2,700","+20,000","+250,000","+7,800,000"
Paul Bartenfeld,-630,-110,+370,"+1,700","+11,000","+120,000","+2,100,000","+94,000,000"


In [34]:
fig6 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[
        "<b>Current Matchup M/L</b><br>Odds for a Single Game", 
        "<b>M/L: Group-Stage Wins</b><br>Odds of Winning at Least <b>X</b> Games"
    ],
    x_title=f"{link_fig}, {link_fig1}, {link_fig2}, {link_fig3}, {link_fig4}, {link_fig5}"
)


go_ml = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in ml_table.columns],
    y=[x.replace(" ", "<br>") for x in ml_table.index],
    z=win_prob_table,
    zmax=max(win_prob_table.max(axis=None), win_prob_table.max(axis=None)),
    zmid=0.5,
    zmin=min(win_prob_table.min(axis=None), win_prob_table.min(axis=None)),
    text=ml_table.applymap(lambda x: "" if math.isnan(float(x)) else x),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

ml_of_cum_x_wins_table.sort_index(axis=0, key=(lambda x: [elo[y] for y in x.values]), inplace=True)
prob_of_cum_x_wins_table.sort_index(axis=0, key=(lambda x: [elo[y] for y in x.values]), inplace=True)
scale_go_ml_cum_x_wins = prob_of_cum_x_wins_table

go_ml_cum_x_wins = go.Heatmap(
    x=[(str(x)) for x in ml_of_cum_x_wins_table.columns],
    y=[x.replace(" ", "<br>") for x in ml_of_cum_x_wins_table.index],
    z=scale_go_ml_cum_x_wins,
    zmax=max(scale_go_ml_cum_x_wins.max(axis=None), scale_go_ml_cum_x_wins.max(axis=None)),
    zmid=0,
    zmin=min(scale_go_ml_cum_x_wins.min(axis=None), scale_go_ml_cum_x_wins.min(axis=None)),
    text=ml_of_cum_x_wins_table.applymap(lambda x: "" if math.isnan(float(x.replace(",", "")))  else x.replace(",", "")),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

fig6.add_trace(go_ml, row=1, col=1)
fig6.add_trace(go_ml_cum_x_wins, row=1, col=2)

fig6.layout.xaxis1.title="<b>Other</b>"
fig6.layout.yaxis1.title="<b>Self</b>"
fig6.layout.xaxis2.title="<b>Number of Total Wins in Group Stage</b>"
fig6.layout.yaxis2.title="<b>Self</b>"

fig6.update_traces(hoverinfo='skip', showscale=False)

fig6.update_layout(
    title=f"<b>Money-Lines</b>, {updated_time}"
)

fig6.show()
fig6.write_html('figures/fig6.html')

In [35]:
ml_table

,Paul Bartenfeld,Roman Ramirez,Evan Sooklal,Aaron Carter,Will Simpson
Paul Bartenfeld,NaN,+145,+425,+610,+970
Roman Ramirez,-145,NaN,+300,+430,+680
Evan Sooklal,-425,-300,NaN,+145,+225
Aaron Carter,-610,-430,-145,NaN,+160
Will Simpson,-970,-680,-225,-160,NaN


In [36]:
# # system optimization

# def optimization1():
#     index = [_ for _ in np.arange(1, 51, 1.0)]
#     columns = [_ for _ in np.arange(2, 11, 1.0)]

#     opt_table = pd.DataFrame(np.zeros([len(index), len(columns)]))
#     opt_table.index = index
#     opt_table.columns = columns

#     for base in opt_table.columns:
#         for K in opt_table.index:
#             (elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, record_table, luck_table, plus_minus_table) = generate_elo(K=K,base=base)
#             num_upsets = 0
#             for i in elo_time_table.index:
#                 prev_elo_slice = prev_elo_time_table.loc[i,:].dropna().values
#                 elo_slice = elo_time_table.loc[i,:].dropna().values
#                 combo_elo = np.array([prev_elo_slice, elo_slice])
#                 is_upset = ((combo_elo[1, 0] - combo_elo[0, 0]) * (combo_elo[0, 1] - combo_elo[0, 0])) > 0
#                 if is_upset:
#                     num_upsets += 1
#             opt_table.loc[K,base] = num_upsets

#     fig = px.imshow(opt_table.T,labels=dict(y="Base", x="K-value", color="Number of Upsets"))#, text_auto=True, aspect="auto")
#     fig.update_layout(
#         title="<b>Total Number of Upsets against K-Value and Base</b>"
#     )
#     fig.show()
#     return opt_table
# opt1 = optimization1()
# fig.write_html("figures/opt1.html")

In [37]:
# # system optimization

# def optimization2():
#     index = [_ for _ in np.arange(1, 51, 1.0)]
#     columns = [_ for _ in np.arange(2, 11, 1.0)]

#     opt_table = pd.DataFrame(np.zeros([len(index), len(columns)]))
#     opt_table.index = index
#     opt_table.columns = columns

#     for base in opt_table.columns:
#         for K in opt_table.index:
#             (elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, record_table, luck_table, plus_minus_table) = generate_elo(K=K,base=base)
#             total_correctness = 0
#             for i in elo_time_table.index:
#                 d_elo_slice = d_elo_time_table.loc[i,:].dropna().values
#                 exp_elo_slice = exp_elo_time_table.loc[i,:].dropna().values
#                 correctness = (np.sign(d_elo_slice) * exp_elo_slice).sum()
#                 total_correctness += correctness
#             opt_table.loc[K,base] = total_correctness

#     fig = px.imshow(opt_table.T,labels=dict(y="Base", x="K-value", color="Total Correctness"))#, text_auto=False, aspect="auto")
#     fig.update_layout(
#         title="<b>Win Percentage Correctness against K-Value and Base</b>"
#     )
#     fig.show()
#     return opt_table
# opt2 = optimization2()
# fig.write_html("figures/opt2.html")

In [38]:
# def norm_minmax(dataframe):
#     return ((dataframe - dataframe.min(axis=None)) / (dataframe.max(axis=None) - dataframe.min(axis=None)))

# norm_opt1 = norm_minmax(opt1)
# norm_opt2 = norm_minmax(opt2)

# fig = px.imshow(norm_opt2.T - norm_opt1.T,labels=dict(y="Base", x="K-value", color="Normalized Optimization"), text_auto=False, aspect="auto")
# fig.update_layout(
#     title="<b>K Value and Base against Both Optimization Functions</b>"
# )
# fig.show()
# fig.write_html("figures/opt_total.html")